In [1]:
import pandas as pd
import numpy as np

In [5]:
df = pd.DataFrame({
    'A': [1, 2, 3,4],
    'B': [4, 5, 6,4],
    'C': [7, 8, 9,4]
})
np_array = df.to_numpy()


In [8]:
np_array.shape

(4, 3)

### Steps
1. Import the files
2. Transform all their features
3. Use the experiment runner and the generate synthetic data class to get info for the experiments
